### 종합실습1
##### Requests 혹은 Selenium 모듈을 이용해 부동산114 모바일 페이지로 접속한 다음, 포항 지역의 아파트 매물 시세 200건을 검색하여 엑셀로 저장해 보세요.
##### 링크 : https://m.r114.com/

### 1. Requests로 크롤링하기

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

#아래에 코드 작성
html = ""
for page_no in range(1, 3):
    url = "https://m.r114.com/Search/m5/search/searchDetailCmd.asp?dqAppendHtml=Y&dqPage="+str(page_no)+"&dqSearchTerm=포항&dqOption=apt&dqSearchType=memul&dqSearchSubType=매매&dqContentSize=100&ranSearchVal=undefined&dqSKString="
    response = requests.get(url, verify = False)
    response.encoding = "UTF-8" #ISO-8859-1으로 인코딩되어 response됨. 이 경우 한글이 깨지므로 UTF-8로 인코딩 변환
    html += response.text

soup = BeautifulSoup(html, "html.parser")
items = soup.select("li")
results = []
for item in items:
    this_result_dict = {
        "아파트명" : item.select(".tit")[0].text,
        "지역" : item.select(".info_wrap")[0].select("span")[0].text,
        "크기" : item.select(".info_wrap")[0].select("span")[1].text,
        "매매가" : item.select(".price")[0].text.replace("매매", "")
    }
    results.append(this_result_dict)

df = pd.DataFrame(results)
df.to_excel("../outputs/포항시 아파트 매매가 동향.xlsx", index = False)
display(df)

### 2. Selenium으로 크롤링하기

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

#아래에 코드 작성
driver = webdriver.Chrome("chromedriver.exe")
driver.get("https://m.r114.com/")
driver.find_element_by_css_selector(".search > a").click()
driver.find_element_by_css_selector("#dqSearchTerm").send_keys("포항")
driver.find_element_by_css_selector(".inp_wrap > .btn_search").click()
driver.find_element_by_css_selector("#body_layout > div:nth-child(2) > div > ul > li.n1 > a").click()

for i in range(0, 20): #스크롤바가 최하단으로가면 추가로 데이터를 로드함. 20번 반복하여 총 200개의 데이터 로드
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    
html = driver.page_source

soup = BeautifulSoup(html, "html.parser")
items = soup.select("#resultUI > li")
results = []
for item in items:
    this_result_dict = {
        "아파트명" : item.select(".tit")[0].text,
        "지역" : item.select(".info_wrap")[0].select("span")[0].text,
        "크기" : item.select(".info_wrap")[0].select("span")[1].text,
        "매매가" : item.select(".price")[0].text.replace("매매", "")
    }
    results.append(this_result_dict)

df = pd.DataFrame(results)
df.to_excel("../outputs/포항시 아파트 매매가 동향.xlsx", index = False)
display(df)